# State Method Design Pattern:

## Some Use Cases:
1. Data Processing Pipeline with Multiple Stages:
- Use Case: In a data pipeline, different stages of data processing may require different operations depending on the data's state (e.g., raw, cleaned, processed). The State pattern can manage transitions between different processing states such as "raw," "cleaned," or "transformed."
- Benefit: The State pattern allows the pipeline to transition seamlessly between stages, applying appropriate logic at each state, and ensuring a consistent workflow for data transformation without complex conditional logic.
2. Real-Time Data Monitoring Systems:
- Use Case: In real-time systems that monitor data streams, the system can transition between states such as "active," "paused," or "error" based on data availability or processing status. The State pattern helps manage these transitions effectively.
- Benefit: The State pattern enables the monitoring system to handle different states dynamically, providing appropriate responses based on the system’s current status, and ensuring better control over real-time data processing.
3. Data Validation Workflow:
- Use Case: In systems that validate data from multiple sources, different validation rules may be applied based on the state of the data (e.g., "pending validation," "valid," "invalid"). The State pattern can define and change validation behaviors based on these states.
- Benefit: The State pattern provides a clean way to handle different validation states, allowing easy maintenance and scalability for validating complex datasets, ensuring that rules are applied correctly at each stage of the validation process.

## 1. Scenario: Traffic Light System
A traffic light system transitions between states (Green, Yellow, and Red) to manage traffic flow. Each state has specific behavior:
- Green: Vehicles move.
- Yellow: Vehicles prepare to stop.
- Red: Vehicles must stop.

### 1.1 Solving with Traditional Method (if-else)

In [4]:
# Traditional approach with if-else
class TrafficLight:
    def __init__(self):
        self.state = "Red"  # Initial state

    def change_state(self):
        if self.state == "Red":
            print("Vehicles must stop.")
            self.state = "Green"
        elif self.state == "Green":
            print("Vehicles can move.")
            self.state = "Yellow"
        elif self.state == "Yellow":
            print("Vehicles should prepare to stop.")
            self.state = "Red"
        else:
            print("Invalid state.")

# Usage
traffic_light = TrafficLight()
traffic_light.change_state()  # Red
traffic_light.change_state()  # Red -> Green
traffic_light.change_state()  # Green -> Yellow
traffic_light.change_state()  # Yellow -> Red

Vehicles must stop.
Vehicles can move.
Vehicles should prepare to stop.
Vehicles must stop.


### 1.2 Solving with State Pattern
### Components in the State Pattern:
- State Interface: Defines a method to change the state.
- Concrete States: Implement state-specific behaviors (Green, Yellow, Red).
- Context: It manages the states and represent the system itself (traffic light).

In [15]:
# State Interface
class TrafficLightState:
    def change_state(self, context):
        raise NotImplementedError("Subclasses must implement this method")

# Concrete State: Red
class RedState(TrafficLightState):
    def change_state(self, context):
        print("Vehicles must stop.")
        context.set_state(GreenState())  # Transition to Green

# Concrete State: Green
class GreenState(TrafficLightState):
    def change_state(self, context):
        print("Vehicles can move.")
        context.set_state(YellowState())  # Transition to Yellow

# Concrete State: Yellow
class YellowState(TrafficLightState):
    def change_state(self, context):
        print("Vehicles should prepare to stop.")
        context.set_state(RedState())  # Transition to Red

# Context: Traffic Light System
class TrafficLight:
    def __init__(self):
        self.state = RedState()  # Initial state

    def set_state(self, state):
        self.state = state  # Set the current state

    def change_state(self):
        self.state.change_state(self)  # Delegate to current state's behavior

# Usage
traffic_light = TrafficLight()
traffic_light.change_state()  # Red
traffic_light.change_state()  # Red -> Green
traffic_light.change_state()  # Green -> Yellow
traffic_light.change_state()  # Yellow -> Red


Vehicles must stop.
Vehicles can move.
Vehicles should prepare to stop.
Vehicles must stop.


## 2. Scenario: Vending Machine System
The vending machine operates in different states like Idle, Has Money, Dispensing Item, Under Maintenance, and Out of Service. Each state dictates the behavior of the machine. For example:
- Idle: The machine is ready to accept money.
- Has Money: The machine waits for the user to select an item.
- Dispensing Item: The machine dispenses the selected item.
- Under Maintenance: The machine cannot accept money or dispense items.
- Out of Service: The machine cannot perform any operations.

### 2.1 Solving with Traditional Method (With if-else or switch-case):

In [8]:
class VendingMachine:
    def __init__(self):
        self.state = "Idle"
    
    def insert_money(self, amount):
        if self.state == "Idle":
            print(f"Money inserted: {amount}")
            self.state = "Has Money"
        elif self.state == "Has Money":
            print("Money already inserted.")
        elif self.state == "Dispensing Item":
            print("Cannot insert money. Dispensing item.")
        elif self.state == "Out of Service":
            print("Machine is out of service.")
        elif self.state == "Under Maintenance":
            print("Machine is under maintenance.")
    
    def select_item(self):
        if self.state == "Idle":
            print("Please insert money first.")
        elif self.state == "Has Money":
            print("Item selected, dispensing now...")
            self.state = "Dispensing Item"
        elif self.state == "Dispensing Item":
            print("Already dispensing an item.")
        elif self.state == "Out of Service":
            print("Machine is out of service.")
        elif self.state == "Under Maintenance":
            print("Machine is under maintenance.")
    
    def dispense_item(self):
        if self.state == "Dispensing Item":
            print("Item dispensed.")
            self.state = "Idle"  # Go back to idle after dispensing
        else:
            print("Cannot dispense item. Select an item first.")
    
    def out_of_service(self):
        self.state = "Out of Service"
        print("Vending machine is out of service.")
    
    def under_maintenance(self):
        self.state = "Under Maintenance"
        print("Vending machine is under maintenance.")
    
    def service_completed(self):
        self.state = "Idle"
        print("Vending machine is ready for use again.")


# Client code:
vending_machine = VendingMachine()

# Insert money
vending_machine.insert_money(5)

# Select item
vending_machine.select_item()

# Dispense item
vending_machine.dispense_item()

# Put machine out of service
vending_machine.out_of_service()

# Attempt to use machine when it's out of service
vending_machine.insert_money(2)

# Under maintenance
vending_machine.under_maintenance()

# Service completed
vending_machine.service_completed()


Money inserted: 5
Item selected, dispensing now...
Item dispensed.
Vending machine is out of service.
Machine is out of service.
Vending machine is under maintenance.
Vending machine is ready for use again.


### Problems with the Traditional Method:
- Hard-Coded Transitions: The if-else statements check the current state and transition based on manual conditions, making it prone to errors when adding new states or behaviors.
- Complexity: As more states and transitions are added, the complexity of the logic increases, making the system harder to maintain and extend.
- Rigid Code: Every change in the state requires modification in multiple parts of the code.
- Code Duplication: Each method (e.g., insert_money, select_item, etc.) repeats the state check logic, leading to redundant code and increased maintenance effort

### 2.2 Solving with the State Pattern:

In [7]:
# State Interface
class VendingMachineState:
    def insert_money(self, context, amount):
        raise NotImplementedError("Subclasses must implement this method.")
    
    def select_item(self, context):
        raise NotImplementedError("Subclasses must implement this method.")
    
    def dispense_item(self, context):
        raise NotImplementedError("Subclasses must implement this method.")

# Concrete State: Idle (Initial State)
class IdleState(VendingMachineState):
    def insert_money(self, context, amount):
        print(f"Money inserted: {amount}")
        context.set_state(HasMoneyState())  # Transition to HasMoneyState

    def select_item(self, context):
        print("Please insert money first.")

    def dispense_item(self, context):
        print("Cannot dispense item. Select an item first.")

# Concrete State: HasMoney
class HasMoneyState(VendingMachineState):
    def insert_money(self, context, amount):
        print("Money already inserted.")
    
    def select_item(self, context):
        print("Item selected, dispensing now...")
        context.set_state(DispensingItemState())  # Transition to DispensingItemState
    
    def dispense_item(self, context):
        print("Cannot dispense item yet. Select an item first.")

# Concrete State: Dispensing Item
class DispensingItemState(VendingMachineState):
    def insert_money(self, context, amount):
        print("Cannot insert money. Dispensing item.")

    def select_item(self, context):
        print("Already dispensing an item.")
    
    def dispense_item(self, context):
        print("Item dispensed.")
        context.set_state(IdleState())  # Transition back to IdleState

# Concrete State: Under Maintenance
class UnderMaintenanceState(VendingMachineState):
    def insert_money(self, context, amount):
        print("Machine is under maintenance. Cannot accept money.")
    
    def select_item(self, context):
        print("Machine is under maintenance. Cannot select items.")
    
    def dispense_item(self, context):
        print("Machine is under maintenance. Cannot dispense items.")

# Concrete State: Out of Service
class OutOfServiceState(VendingMachineState):
    def insert_money(self, context, amount):
        print("Machine is out of service.")
    
    def select_item(self, context):
        print("Machine is out of service.")
    
    def dispense_item(self, context):
        print("Machine is out of service.")

# Context: Vending Machine
class VendingMachine:
    def __init__(self):
        self.state = IdleState()  # Initial state

    def set_state(self, state):
        self.state = state  # Set the new state

    def insert_money(self, amount):
        self.state.insert_money(self, amount)

    def select_item(self):
        self.state.select_item(self)

    def dispense_item(self):
        self.state.dispense_item(self)

# Usage
vm = VendingMachine()
vm.insert_money(5)  # Transition to HasMoneyState
vm.select_item()  # Transition to DispensingItemState
vm.dispense_item()  # Transition back to IdleState

# Switching to Under Maintenance
vm.set_state(UnderMaintenanceState())
vm.insert_money(5)  # Machine is under maintenance
vm.select_item()  # Machine is under maintenance


Money inserted: 5
Item selected, dispensing now...
Item dispensed.
Machine is under maintenance. Cannot accept money.
Machine is under maintenance. Cannot select items.


### How State Method Pattern solve problems:
- Avoids complex conditionals: No need for if-else or switch-case to manage states.
- Encapsulates behavior: Each state has its own class for specific behavior.
- Simplifies state transitions: Transitions handled within state classes, making flow clear.
- Improves flexibility: New states can be added easily without affecting other code.
- Decouples logic: Context delegates behavior to the current state.
- Enhances scalability: New states can be added without modifying existing code.
- Reduces Code Duplication: Each state-specific behavior is encapsulated in its own class, reducing repetitive logic and making maintenance easier.

## State Method Pattern and Strategy Method seems similar but they are different:
### State Pattern:
The behavior of an object changes based on its current state, and the state transitions between predefined states. The object repeats certain behaviors depending on which state it’s in. Each state has specific logic, and state transitions drive the flow.
- Example: In a vending machine, the state transitions from "Idle" to "Has Money" to "Dispensing Item", and each state has distinct behavior.

### Strategy Pattern:
The behavior changes dynamically by selecting different strategies (or algorithms). There is no repetitive internal state, and the change is based on switching strategies rather than transitioning states.
- Example: A payment system dynamically selects a strategy like "CreditCardStrategy" or "CashStrategy" based on the user’s choice. There are no repeated internal transitions, just swapping strategies.